# Similar Artists to Artists I Already Like (Run every February, and August)

In [2]:
import csv
import requests
from typing import List, Dict, Optional, Set
from time import sleep
from datetime import datetime

class LastFMAPI:
    def __init__(self, api_key: str, username: str, rate_limit_delay: float = 0.25):
        self.api_key = api_key
        self.username = username
        self.base_url = "http://ws.audioscrobbler.com/2.0/"
        self.rate_limit_delay = rate_limit_delay
        
    def _make_request(self, params: Dict) -> Optional[Dict]:
        try:
            # Handle rate limit by checking headers for remaining requests
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()

            # Check for rate limit info in the response headers
            remaining = int(response.headers.get('X-RateLimit-Remaining', 1))
            if remaining == 0:
                reset_time = int(response.headers.get('X-RateLimit-Reset', 0))
                wait_time = reset_time - int(datetime.now().timestamp())
                print(f"Rate limit hit, waiting for {wait_time} seconds...")
                sleep(wait_time + 1)  # wait for the reset time plus 1 second for safety
                response = requests.get(self.base_url, params=params)  # retry after waiting
                response.raise_for_status()

            return response.json()

        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            if response.status_code == 429:
                print("Rate limit exceeded, increasing delay.")
                self.rate_limit_delay *= 2
            return None

    def get_scrobbles(self, page: int, limit: int = 200) -> List[Dict]:
        params = {
            'method': 'user.getrecenttracks',
            'user': self.username,
            'api_key': self.api_key,
            'format': 'json',
            'page': page,
            'limit': limit,
            'from': int(datetime(2022, 1, 1).timestamp())
        }
        data = self._make_request(params)
        if data and 'recenttracks' in data:
            return data['recenttracks'].get('track', [])
        return []

    def get_similar_artists(self, artist_name: str, limit: int = 5) -> List[str]:
        params = {
            'method': 'artist.getsimilar',
            'artist': artist_name,
            'api_key': self.api_key,
            'format': 'json',
            'limit': limit
        }
        data = self._make_request(params)
        if data and 'similarartists' in data:
            return [artist['name'] for artist in data['similarartists'].get('artist', [])]
        return []

def collect_unique_artists(api: LastFMAPI, max_pages: int = 100) -> Set[str]:
    unique_artists = set()
    total_tracks = 0
    
    print("Phase 1: Collecting unique artists...")
    for page in range(1, max_pages + 1):
        tracks = api.get_scrobbles(page)
        if not tracks:
            break
            
        for track in tracks:
            artist = track.get('artist', {}).get('#text', 'Unknown')
            if artist != 'Unknown':
                unique_artists.add(artist)
        
        total_tracks += len(tracks)
        print(f"Processed page {page}, total tracks: {total_tracks}, unique artists: {len(unique_artists)}")
        
        # Check if we've gone past 2022
        if tracks and 'date' in tracks[-1]:
            last_timestamp = int(tracks[-1]['date']['uts'])
            if last_timestamp < datetime(2022, 1, 1).timestamp():
                print("Reached pre-2022 tracks, stopping...")
                break
    
    return unique_artists

def export_similar_artists(api_key: str, username: str, output_file: str = 'data/artist_recommendations.csv', max_pages: int = 100):
    api = LastFMAPI(api_key, username)
    
    # Phase 1: Collect all unique artists
    unique_artists = collect_unique_artists(api, max_pages)
    
    # Phase 2: Get similar artists for each unique artist
    print(f"\nPhase 2: Finding similar artists for {len(unique_artists)} unique artists...")
    
    try:
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['Artist', 'Similar Artists'])
            writer.writeheader()
            
            for i, artist in enumerate(unique_artists, 1):
                similar_artists = api.get_similar_artists(artist)
                writer.writerow({
                    'Artist': artist,
                    'Similar Artists': ', '.join(similar_artists)
                })
                
                if i % 10 == 0:
                    print(f"Processed similar artists for {i}/{len(unique_artists)} artists")
        
        print(f"\nExport complete! Processed {len(unique_artists)} unique artists.")
        
    except Exception as e:
        print(f"Fatal error during export: {e}")

if __name__ == "__main__":
    API_KEY = '74a510ecc9fc62bf3e0edc6adc2e99f9'
    USERNAME = 'Strusz_Music'
    
    export_similar_artists(API_KEY, USERNAME)
